In [1]:
#purpose is to generate images in the steps below, first collecting some files
'''
Paths and file needs:
*imglams and spitzer_conversions are excel files, right now I have it so you need to put it as same directory as your code (but could later maybe just give it a path to go to - would be smarter)
*paths to images and data in general
'''
#now the steps
'''
1) read in all the data by noting all the paths to given spitzer and hubble images
2) loop through all the data, read it in, convert units
3) cutout all the data as appropriate
3) create a loop or otherwise hardcode going through all the combinations of convolutions of images by hand...
4) regrid all the images
5) de-extinct all the images
6) create apertures as appropriate for all the knots
7) perform relevant analyses: e.g. taking ratio and then finding EDFs, summing up the intensities of each knot for noting and saving
'''

#just to check python version - should be 3.7.4
from platform import python_version
print(python_version())

#importing libraries
from astropy.io import fits
from astropy.convolution import convolve, Gaussian2DKernel, Box2DKernel
from astropy.nddata import Cutout2D
from astropy.wcs import WCS

import glob
import itertools
import matplotlib 
%matplotlib inline
# matplotlib.use('Agg') #invokved b/c just plain matplotlib was insufficient
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

import pims
import trackpy as tp
import os

#this part is unnecessary...in jupyter...point was to test blocks of code
#switches for the three different parts of this code
# switch1 = 'on' #convolving images [needed to put it on for switch 3 at min...need to figure out other solution, eh]
# switch1b = 'on' #regridding...
# switch2 = 'on' #solving equations
# switch3 = 'on' #plotting / graphics of solutions

3.8.5


In [2]:
# #finding the path to every fits images in a directory
def im_name_finder(path, file_type):
    #Using glob (it's a unix command similar to ls)
    #WARNING: using recursive=True...depending how many images you use this could be very slow, it's recommended not to have too many subfolders
    #if needed, some example code is commented towards the latter half of this code that could help make an alternative
    all_names = glob.glob(path, recursive=True)

    #IMPORTANT: Using "fit" here because it is inclusive of both fits and FIT...some files end in "FIT" and need to be included
    #using s.lower() include uppercase names
    im_names = [s for s in all_names if 'fit' in s.lower()]

    return im_names


'''now convolve my image with a PSF of the image we're projecting ONTO
an approx PSF can be found by assuming a 2D Gaussian func with a width (a FWHM) of the diffrac limit
that is the st dev of the Gaussian is about the st dev is about = lambda/D
a list of PSFs are found on https://docs.astropy.org/en/stable/convolution/kernels.html

Notes:
FIRST: always must convert hdu1_pixtorad to radians! It's inconsistent otherwise, and lambda/D is generally in radians

what we're using for the gaussian width is the FWHM, not the radius of the first ring of the diffraction pattern,
so it's 1.2 not 1.22 times lambda/D

D is 85 cm for spitzer
D is 2.4 m for hubble
'''

def im_conv(D, hdu_pix_torad, hdu_dat, lam, kern):
    #gaussian kernel
    if kern == 'gauss':
        #update: usually cannot find wavelength but these headers are well-labeled    
        #finding angular resolution...the FWHM of our Gaussian PSF
        res = 1.2 * lam / D         #resolution in radians
        res = res / hdu_pix_torad        #so converting to pixels

        #finding PSF and then calculating the convolution of our image and the PSF of the image we're projecting onto
        kernel = Gaussian2DKernel(res)

    #box kernel
    if kern == 'box':
        kernel = Box2DKernel(16.)

    hdu_conv = convolve(hdu_dat, kernel)
    return hdu_conv

# In[27]:

#setting up a new fits file to be saved and viewed in DS9
#primarily to save the image we reprojected, but can also be used to save the convolved images
def fits_saver(array, wcs_header, name, save_path):
    '''
    array is a 2d array of data - could be from reprojecting one image onto another or from convolution
    wcs_header is a header containing the wcs coords of the image that we projected onto or of the orig image (if from the convolution)
    name is the path to some image you're using. It will get string split at the / character, and the func only takes the last element of that splitting
    save_path is the folder you want to save to...recommended to also add something to the start of the images names to make it clear what you did to them (e.g. 'Regridded/regrid_')
    '''

    #creating a new file and adding the reprojected array of data as well as the WCS that we projected onto
    hdu_new = fits.PrimaryHDU(array, header=wcs_header)
    hdul = fits.HDUList([hdu_new])
    
    #saving the file
    if name.find('FIT') == -1: #needed if file end incorrect
        new_filename = name.split('/')[-1]  #grabs the file name we were using from before
        hdul.writeto(save_path+new_filename, overwrite=True)
    else:
        name_fixfit = name[:-3] + 'fits'
        new_filename = name_fixfit.split('/')[-1]  #grabs the file name we were using from before
        hdul.writeto(save_path+new_filename, overwrite=True)
        
    return (save_path+new_filename)

This code uses TrackPy...

http://soft-matter.github.io/trackpy/v0.5.0/tutorial/custom-feature-detection.html

In [51]:
# path = '../../Montage_results/n2071_headercut/*drz.fits' # #using ** will grab all files even in subdirectories WARNING takes longer
# im_names_n2071 = sorted(im_name_finder(path, 'fit')) #im_finder is basically glob.glob
# im_names_n2071 = [i.replace('\\', '/') for i in im_names_n2071]
im_names_n2071 = ['../scaling_for_motions/160_epoch1_scaled.fits', '../scaling_for_motions/160_epoch2_synth_scaled.fits']

print(im_names_n2071)

['../scaling_for_motions/160_epoch1_scaled.fits', '../scaling_for_motions/160_epoch2_synth_scaled.fits']


In [52]:
hdu_list = [fits.open(i) for i in im_names_n2071]

#initializing some lists to be used
hdu_pix_list = []
hdu_pixtorad_list = []
# hdu_fnu_list = []
hdu_lam_list = []
# hdu_flam_list = []
# hdu_bw_list = []
hdu_data_list = []
hdu_header_list = []

count = 0
for hdu_data in hdu_list:   
    #reading in conversions
#     hdu_pix_list.append(hdu_data[0].header['D001SCAL'])  #D001SCAL is the keyword for Hubble images, in sr
#     hdu_pixtorad_list.append(hdu_pix_list[count] / 206265.)
    # hdu_fnu_list.append(hdu_units[0].header['PHOTFNU'])
#     hdu_lam_list.append(hdu_data[0].header['PHOTFLAM'])
#     hdu_flam_list.append(hdu_list[0].header['PHOTFLAM'])
#     hdu_bw_list.append(hdu_list[0].header['PHOTBW'])

    #reading in data for general use  and header for wcs
    #converting by times by flam * bw from e-/sec...should get units of erg/cm^2/sec as above
    hdu_data_list.append(hdu_data[0].data) # * hdu_list[0].header['PHOTFLAM'] * hdu_list[0].header['PHOTBW'])
    hdu_header_list.append(hdu_data[0].header)
    
    count+=1
    


In [61]:
from scipy import ndimage
from skimage import morphology, util, filters

@pims.pipeline
def preprocess_foam(img):
    """
    Apply image processing functions to return a binary image
    """
    # Crop the pictures as for raw images.
    # Apply thresholds
    img_crop = img
    print(img_crop, img_crop.shape)
    adaptive_thresh = filters.threshold_local(img_crop,1)
    idx = img_crop > adaptive_thresh
    idx2 = img_crop < adaptive_thresh
    img_crop[idx] = 0
    img_crop[idx2] = 255
    img_crop = ndimage.binary_dilation(img_crop)
    img_crop = ndimage.binary_dilation(img_crop)
    return util.img_as_int(img_crop)

#plotting resulting image
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.utils.data import get_pkg_data_filename
from astropy.visualization import ZScaleInterval, ImageNormalize, PercentileInterval
from astropy.visualization.stretch import SinhStretch, AsinhStretch, LogStretch
from astropy.visualization.wcsaxes import WCSAxesSubplot
from astropy.wcs import WCS
from astropy.wcs.utils import skycoord_to_pixel
import matplotlib.ticker
from matplotlib.ticker import LogFormatter, LogLocator, FixedLocator, FixedFormatter

f = tp.locate(hdu_data_list[0], 15)
tp.annotate(f, hdu_data_list[0])

sys.exit()

fig = plt.figure(figsize=(20,20))
wcs = WCS(hdu_header_list[0])
ax = plt.subplot(projection=wcs)

#plotting
interval = PercentileInterval(99)
data_interval = interval.get_limits(hdu_data_list[0])
ds9_min = -1.64092e-17
ds9_max = 3.19425e-16
norm = ImageNormalize(stretch=AsinhStretch(), vmin=data_interval[0], vmax=data_interval[1])
# norm = ImageNormalize(stretch=SinhStretch(), vmin=ds9_min, vmax=ds9_max)
# norm = ImageNormalize(stretch=AsinhStretch(), vmin=-8e-18, vmax=8e-18)
# norm = ImageNormalize(stretch=LogStretch(), vmin=0.1e-18, vmax=100e-18)

id_example = 0
# frames = preprocess_foam(hdu_data_list[0]) #   pims.open(os.path.join(datapath, prefix + '*.tif')))
im = ax.imshow(hdu_data_list[0], norm=norm, origin='lower', cmap='Greens_r')
# hops_sources = [ax.scatter(c_pair[0], c_pair[1], color='gold', marker='+', s=100) for c_pair in coord_pix_list]

#general formatting
ax.coords.grid(True, color='black', ls='solid', linewidth=0.75) #adding gridlines
ax.coords[0].set_axislabel('Right Ascension (J2000)', fontsize=30)
ax.coords[1].set_axislabel('Declination (J2000)', fontsize=30)   
ax.tick_params(axis='x', labelbottom=True, labeltop=False, labelright=False)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)
ax.invert_yaxis() #done because it's nicer with declination increasing
ax.invert_xaxis() #done because it's nicer with declination increasing

#colorbar, see 3rd answer from https://stackoverflow.com/questions/18195758/set-matplotlib-colorbar-size-to-match-graph
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
cbar = plt.colorbar(im, cax=cax) 

#color bar label and tick labels
# sub_labels = [2]
# cbar.locator = LogLocator(base=10, subs=sub_labels)
cbar.ax.tick_params(labelsize=20)
cbar.update_ticks()
cbar.ax.yaxis.get_offset_text().set_fontsize(0)
cbar.set_label(label=r'$\rm Intensity~(x~{10}^{-18}~erg/s/{cm}^{2}/pix)$', size=20)
# cbar_tickfont = [cbar.ax.set_yticklabels(labels=cbar.ax.get_yticklabels())[i].set_fontweight('normal') for i in range(len(cbar.ax.get_yticklabels()))]
# plt.savefig('f164n_epoch_2.png', dpi=300, bbox_inches="tight")
plt.show()

# fits_saver(frames, hdu_header_list[0], '160_epoch1.fits', '') #max determined from fits file

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at C:\Users\arubi\anaconda3\lib\site-packages\trackpy\refine\center_of_mass.py (362)[0m
[1m
File "..\..\..\..\..\..\..\..\anaconda3\lib\site-packages\trackpy\refine\center_of_mass.py", line 362:[0m
[1mdef _numba_refine_2D_c(raw_image, image, radiusY, radiusX, coords, N,
    <source elided>
    # Column indices into the 'results' array
[1m    MASS_COL = 2
[0m    [1m^[0m[0m

This error may have been caused by the following argument(s):
- argument 0: Unsupported array dtype: >f8

This error may have been caused by the following argument(s):
- argument 0: Unsupported array dtype: >f8


In [21]:
#our plotting function
def implot(data, w, wcscond, vmax_p):
    fig = plt.figure()
    
    if  wcscond == True:
        fig.add_subplot(111, projection=w)
    else:
        fig.add_subplot(111)
    
    #for christmas turn on GnRd
    #plt.cm.get_cmap('Blues', 6) is another option
    #can also use RdBu...
    #otherwise just use plt.cm.viridis b/c it works
    plt.imshow(data, origin='lower', cmap=plt.cm.viridis, vmin =0, vmax=vmax_p)
    plt.xlabel('RA')
    plt.ylabel('Dec')


In [23]:
%matplotlib inline